In [ ]:
pip install neo4j

In [ ]:
from neo4j import GraphDatabase
import json

class Neo4jQueries:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def run_query(self, query, params=None):
        with self.driver.session() as session:
            result = session.run(query, params)
            return [record.data() for record in result]

    def top_papers_per_conference(self):
        query = """
        MATCH (p:Paper)-[:Cited_by]->(citing:Paper)
        WITH p, count(citing) AS citation_count
        MATCH (p)-[:Published_in]->(ce:Conference_Edition)-[:Of]->(conf:Conference)
        WITH conf, p.title AS paper_name, citation_count
        ORDER BY conf, citation_count DESC
        WITH conf.Name AS conference, collect({paper: paper_name, citations: citation_count})[0..3] AS top_papers
        RETURN conference, top_papers
        """
        return self.run_query(query)

    def community_authors(self):
        query = """
        MATCH (a:Author)-[:Writes]->(p:Paper)-[:Published_in]->(ce:Conference_Edition)-[:Of]->(conf:Conference)
        WITH conf, a, count(DISTINCT ce) AS editions_count
        WHERE editions_count >= 4
        RETURN conf.Name AS conference, collect(DISTINCT a.Name) AS community_authors
        ORDER BY conference
        """
        return self.run_query(query)

    def journal_impact_factors(self, year=2022):
        query = """
        WITH $year AS y
        // Step 1: Get citations in year y for papers published in (y-1) or (y-2)
        MATCH (citing:Paper)-[:Cited_by]->(p:Paper)-[:Published_in]->(j:Journal)
        WHERE p.year IN [y-1, y-2] AND citing.year = y
        WITH j, count(citing) AS total_citations, $year as y
        // Step 2: Count total papers published in (y-1) and (y-2)
        MATCH (p:Paper)-[:Published_in]->(j)
        WHERE p.year IN [y-1, y-2]
        WITH j, total_citations, count(p) AS total_papers_published
        WHERE total_papers_published > 0
        // Step 3: Compute impact factor
        RETURN j.ID, j.Name AS journal, total_citations, total_papers_published, 
        total_citations * 1.0 / total_papers_published AS impact_factor
        ORDER BY impact_factor DESC
        """
        return self.run_query(query, {"year": year})

    def calculate_h_indices(self):
        query = """
        MATCH (a:Author)-[:Writes]->(p:Paper)
        OPTIONAL MATCH (p)-[:Cited_by]->(citing:Paper)
        WITH a, p, count(citing) AS citation_count
        ORDER BY a.Name, citation_count DESC // Sort papers by citations in descending order

        WITH a.Name AS author, collect(citation_count) AS citations_list
        UNWIND range(1, size(citations_list)) AS h // Generate index positions (h)
        WITH author, h 
        WHERE citations_list[h-1] >= h // Check if the h-th paper has at least h citations

        RETURN author, max(h) AS h_index
        ORDER BY h_index DESC
        """
        return self.run_query(query)

    

In [ ]:
def main():
    # Replace with your Neo4j credentials
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "pass1234"
    
    try:
        queries = Neo4jQueries(uri, user, password)
        
        print("\n======= Analysis Queries =======")
        
        print("\n--- Top Papers Per Conference ---")
        top_papers = queries.top_papers_per_conference()
        print(json.dumps(top_papers[:5], indent=2))  # Print first 5 for brevity
        
        print("\n--- Community Authors ---")
        community_authors = queries.community_authors()
        print(json.dumps(community_authors[:5], indent=2))  # Print first 5 for brevity
        
        print("\n--- Journal Impact Factors (2022) ---")
        impact_factors = queries.journal_impact_factors(2022)
        print(json.dumps(impact_factors[:5], indent=2))  # Print first 5 for brevity
        
        print("\n--- H-Indices for Authors ---")
        h_indices = queries.calculate_h_indices()
        print(json.dumps(h_indices[:10], indent=2))  # Print top 10 for brevity
        
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        queries.close()

if __name__ == "__main__":
    main()